# By providing examples of extraction, get LLM to follow and extract info in JSON

## Pip installs (if required)

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Imports

In [2]:
%cd ..
from src import *
%cd experiments/

c:\Users\Zain\Documents\Github\University


c:\Users\Zain\Documents\Github\University\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\Zain\Documents\Github\University\experiments


c:\Users\Zain\Documents\Github\University\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import json
import os
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import unicodedata
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "OpenAIApp"
encoding: str = "utf-32"
class Extractor:
    """Given example text and extraction, query all chunks of text and collect as JSON. Store in experiments folder"""
    def __init__(self, folder: str):
        self.folder = folder
    def run(self):
        ex: list = self._load_text()
        sys: str = self._makeQuery(ex)
        num: int = self._getLenChunks()
        for i in range(num):
            text: str = self._readChunk(i)
            # print(text)
            response: dict = self._queryLang(sys, text)
            self._storeText(response, f"chunk{i}.txt")
        return text
    def _load_text(self) -> list:
        """Return both example text and example extracted data from it"""
        with open(f"experiments/{self.folder}/examples/sample_text.txt", "r", encoding = "ascii") as file:
            example = file.read()
            example = unicodedata.normalize('NFC', example)
        with open(f"experiments/{self.folder}/examples/extracted_text.txt", "r", encoding = "ascii") as file:
            extract = file.read()
            extract = unicodedata.normalize('NFC', extract)
        return [example, extract]
    def _makeQuery(self, list) -> str:
        """Use list of 2 texts to form system message"""
        content = f"""
Here is an example of some text in which definitions, lemmas, theorems, propositions and corollaries are extracted:

{list[0]}

And here is the extracted result in json format:

{list[1]}

You will use the examples given above to perform the same task on the text provided in the user message.
"""
        return content
    def _getLenChunks(self) -> int:
        """Return how many times to read chunks"""
        return len(os.listdir(f"experiments/{self.folder}/chunks/"))
    def _readChunk(self, chunk: int) -> str:
        """Read in chunk of text"""
        with open(f"experiments/{self.folder}/chunks/chunk{chunk}.txt", "r", encoding = "ascii") as file:
            content = file.read()
            content = unicodedata.normalize('NFC', content)
            return content
    def _queryLang(self, sys: str, user: str) -> dict:
        """Query Langchain and return json of response"""
        # print(sys)
        # print(type(user))
        llm = ChatOpenAI(model = "gpt-4o-mini")
        md: str = rf"""{sys}"""
        md = json.dumps(md.replace("{", "{{").replace("}", "}}"), indent = 4)
        usermd: str = rf"""{user}"""
        usermd = json.dumps(usermd.replace("{", "{{").replace("}", "}}"), indent = 4)
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", md),
                ("user", "{input}")
            ]
        )
        chain = prompt|llm # combines both
        response = chain.invoke({"input": usermd})
        return response.content
    def _storeText(self, text: str, filename: str) -> None:
        """Temporarily store as text"""
        os.makedirs(f"experiments/{self.folder}/output/", exist_ok = True)
        with open(f"experiments/{self.folder}/output/{filename}", "w") as file:
            file.write(text)
    def _storeQuery(self, dict, filename:str) -> None:
        """Store langchain response as .json"""
        os.makedirs(f"experiments/{self.folder}/output/", exist_ok = True)
        with open(f"experiments/{self.folder}/output/{filename}", "w") as file:
            json.dump(dict, file, indent = 4)

## Setup

In [4]:
def pipeline(filename: str):
    """Runs full pipeline"""
    docIngest(filename).run()
    mdCombine(filename).run()
    getChunks(filename).run()
    Extractor(filename).run()

In [5]:
from datetime import datetime
current_datetime = datetime.now()

# Format it as a string
formatted_datetime: str = current_datetime.strftime("%Y%m%d%H%M%S")
name: str = "experiment" + formatted_datetime
print(f"Folder found in experiments/{name}")
pipeline(name)

Folder found in experiments/experiment20241007164528


UnicodeDecodeError: 'ascii' codec can't decode byte 0xef in position 1281: ordinal not in range(128)

# Code Here

In [9]:
# Squasher to combine all output chunks into 1